In [58]:
#PART1

In [59]:
import pandas as pd
import torch
import numpy as np

In [60]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.values

array([[ 24,  51,  44, ...,  27,   0,   3],
       [ 22,  31, 214, ...,   0,   0,   2],
       [ 84, 126, 239, ...,  24,   8,   9],
       ...,
       [268, 358, 277, ...,  47,   7,   5],
       [ 81, 116,  90, ...,   9,   0,   2],
       [118, 156, 197, ...,   0,   0,   5]])

In [63]:
trainX = torch.from_numpy(train_df.values[:,:-1]).to(torch.float64)
trainY = torch.from_numpy(train_df.values[:,-1:]).to(torch.float64)

testX = torch.from_numpy(test_df.values[:,:-1]).to(torch.float64)
testY = torch.from_numpy(test_df.values[:,-1:]).to(torch.float64)
trainX

tensor([[ 24.,  51.,  44.,  ...,  61.,  27.,   0.],
        [ 22.,  31., 214.,  ...,   9.,   0.,   0.],
        [ 84., 126., 239.,  ...,  74.,  24.,   8.],
        ...,
        [268., 358., 277.,  ..., 107.,  47.,   7.],
        [ 81., 116.,  90.,  ...,  36.,   9.,   0.],
        [118., 156., 197.,  ...,  19.,   0.,   0.]], dtype=torch.float64)

In [64]:
trainY

tensor([[3.],
        [2.],
        [9.],
        ...,
        [5.],
        [2.],
        [5.]], dtype=torch.float64)

In [65]:
coef = torch.tensor([
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040], # POS_50_59_CP
        [0.0300], # POS_60_69_CP
        [0.0300],
        [0.0300],
        [0.0300]
], dtype=testX.dtype)
coef

tensor([[0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0300],
        [0.0300],
        [0.0300],
        [0.0300]], dtype=torch.float64)